In [1]:
!pip install pyomo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 3.7 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 


In [2]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [11]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [4]:
n = int(input())
coordinates = []
for i in range(n):
    coordinates.append(list(map(int,input().split())))
x_axis = []
for i in (range(len(coordinates))):
  x_axis.append(coordinates[i][0])
y_axis = []
for i in (range(len(coordinates))):
  y_axis.append(coordinates[i][1])

3
2 5
3 3
3 5


In [5]:
model = pyo.ConcreteModel()

model.x = pyo.Var(domain=pyo.NonNegativeIntegers)
model.y = pyo.Var(domain=pyo.NonNegativeIntegers)
x = model.x
y = model.y

model.new_var = pyo.VarList(domain=pyo.Integers)
for i in (range(2 * len(x_axis))):
  model.new_var.add()

model.c = pyo.ConstraintList()
for i in range(len(x_axis)):
  model.c.add(expr=  x - x_axis[i] <= model.new_var[i+1])
  model.c.add(expr= -1 * (x - x_axis[i]) <= model.new_var[i+1])
for i in range(len(x_axis), 2*len(x_axis)):
  j = i - len(x_axis)
  model.c.add(expr=  y - y_axis[j] <= model.new_var[i+1])
  model.c.add(expr= -1 * (y - y_axis[j]) <= model.new_var[i+1])

In [6]:
model.obj = pyo.Objective(expr = sum(model.new_var) , sense = minimize)
opt = SolverFactory('glpk')
opt.solve(model)
model.pprint()
x_value = pyo.value(x)
y_value = pyo.value(y)
print('optimal coordinate = (',x_value, ',', y_value, ')')

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    new_var_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}

3 Var Declarations
    new_var : Size=6, Index=new_var_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :   1.0 :  None : False : False : Integers
          2 :  None :   0.0 :  None : False : False : Integers
          3 :  None :   0.0 :  None : False : False : Integers
          4 :  None :   0.0 :  None : False : False : Integers
          5 :  None :   2.0 :  None : False : False : Integers
          6 :  None :   0.0 :  None : False : False : Integers
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   3.0 :  None : False : Fal

In [10]:
sum_ = 0 
for i in range(len(x_axis)):
  sum_+= abs(x_value - x_axis[i]) + abs(y_value - y_axis[i])
print('optimal cost value = ',sum_)

optimal cost value =  3.0
